In [1]:
import re
## if you use this notebook, you can get the treebank files here.
#!wget https://raw.githubusercontent.com/gregorycrane/gAGDT/master/data/xml/tlg0012.tlg001.perseus-grc1.tb.xml
#!wget https://raw.githubusercontent.com/gregorycrane/gAGDT/master/data/xml/tlg0012.tlg002.perseus-grc1.tb.xml
#fname1 = 'tlg0012.tlg001.perseus-grc1.tb.xml'
#fname2 = 'tlg0012.tlg001.perseus-grc2.tb.xml'

fname1 = '/Users/gcrane/github/gAGDT/data/xml/tlg0012.tlg001.perseus-grc1.tb.xml'
fname2 = '/Users/gcrane/github/gAGDT/data/xml/tlg0012.tlg002.perseus-grc1.tb.xml'

textlines = {}
prepositions = {}
curcites ={}
def gettreebankdata(fname):
    f = open(fname)
    for l in f:
        l = re.sub("'",'’',l)
        m = re.search('id="([^"]+)',l)
        if(m and re.search('<sentence',l)):
            sentid = m[1]

        if( not re.search('lemma=',l)):
            continue
        m = re.search('lemma="(["]+)',l)
        if(m):
            curlem = m[1]
    
        m = re.search('id="([^"]+)',l)
        if(m):
            if(re.search('<sentence',l)):
                sentid = m[1]
            if(re.search('<word',l)):
                wordid = sentid + '.' + m[1]
    
        m = re.search('form="([^"]+)',l)
        if(m):
            curform = m[1]
            curform = re.sub("'",'’',curform)
    
        m = re.search('postag="([^"]+)',l)
        if(m):
            curpostag = m[1]
            if(curpostag[0]=='r'):
                if(curform in prepositions):
                    prepositions[curform] = prepositions[curform] + 1
                else:
                    prepositions[curform] = 1
    
        m = re.search('head="([^"]+)',l)
        if(m):
            curhead = sentid + '.' + m[1]
    
        m = re.search('cite="urn:cts:greekLit:tlg0012.(tlg00[^"]+)',l)
        if(m):
            curcite = m[1]
            curkey = curcite + ':' + curform
            if(curkey in curcites):
                m = re.search(':([0-9]+)$',curkey)
                if(m):
                    index = int(curcite) + 1
                else:
                    index = 2
                curkey = re.sub(':([0-9]+)$','',curkey)
                curkey = curkey + ':' + str(index)
            if(curcite in textlines):
                textlines[curcite] = textlines[curcite] + ' ' + curform
            else:
                textlines[curcite] = curform
                #print(curkey)
            curcites[curkey] = 1
    f.close()
    
    
gettreebankdata(fname1)
gettreebankdata(fname2)
print('donetb')

donetb


In [2]:

from fuzzywuzzy import fuzz
from greek_accentuation.characters import *
from greek_accentuation.syllabify import *
from greek_accentuation.accentuation import *
badbooks = {}
booklines = {}

for i in range(1,25):
    badbooks[i] = 0

for i in range(1,25):
    booklines[i] = 0

def nodiaeresis(s):
    rs = ''
    for foo in s:
        if(foo == 'ϊ'):
            rs = rs + 'ι'
        elif(foo == 'ΐ'):
            rs = rs + 'ί'
        elif(foo == 'ῗ'):
            rs = rs + 'ῖ'
        elif(foo == 'ΰ'):
            rs = rs + 'ύ'
        elif(foo == 'ϋ'):
            rs = rs + 'υ'
        elif(foo == 'ῢ'):
            rs = rs + 'ὺ'
        elif(foo == 'ῧ'):
            rs = rs + 'ῦ'
        else:
            rs = rs + foo
    return(rs)

def getbestfit(tkey):
    args = tkey.split(':')
    curcit = re.sub(':[0-9]+$','',tkey)
    curcit = re.sub(':[^:]+$','',curcit)
    if(not curcit in textlines):
        print('failed',curcit,tkey)
        return '',0
    bestfuz = 0
    bests = ''
    for foo in textlines[curcit].split():
        test = nodiaeresis(foo)
        r = fuzz.ratio(test,args[2])
        if(r > bestfuz):
            bestfuz = r
            bests = foo
        
    return bests,bestfuz
metercites = {}
def checkwords(fname):
    f = open(fname)
    badforms = {}
    badcount = 0
    tokencount = 0
    equivs2 = {}
    equivs1 = {}

    equivs2['ἔγωγ’'] ="ἐγώ γ’"
    equivs2['ἔγωγε'] ="ἐγώ γε"
    equivs2['ἔγωγέ'] ="ἐγώ γε"
    equivs2['ἔμοιγ’'] ="ἐμοί γ’"
    equivs2['ἔμοιγε'] ="ἐμοί γε"
    equivs2['ἔμοιγέ'] ="ἐμοί γε"
    
    
    equivs1['Ἠΰτε'] ="ἠΰτε"
    equivs1['Ἠΰτ’'] ="ἠΰτ’"
    equivs1['Ἠύτε'] ="ἠΰτε"
    equivs1['Ἠύτ’'] ="ἠΰτ’"
    equivs2['Λυκίηνδε'] ="Λυκίην δ[έὲ]"
 

    equivs2['οἶκονδε'] ="οἶκον δὲ"
    equivs2['πεδίονδε'] ="πεδίον δ[έὲ]"
    equivs2['πόλινδε'] ="πόλιν δὲ"
    
    
    equivs2['Οὐδέ'] ="οὐ δέ"
    equivs2['Οὐδέ'] ="οὐ δέ"
    equivs2['οὐδέ'] ="οὐ δέ"
    equivs2['Οὐδὲ'] ="οὐ δὲ"
    equivs2['οὐδὲ'] ="οὐ δὲ"
    
    equivs2['οὐδ’'] ="οὐ δ’"
    equivs2['Οὐδ’'] ="οὐ δ’"
    equivs2['οὔτε'] ="οὔ τε"
    equivs2['οὔτ’'] ="οὔ τ’"
    equivs2['οὔθ’'] ="οὔ θ’"
    
    equivs2['μηδέ'] ="μη δέ"
    equivs2['μηδὲ'] ="μη δὲ"
    equivs2['μηδ’'] ="μη δ’"
    equivs2['μήτε'] ="μή τε"
    equivs2['μήτ’'] ="μή τ’"
    equivs2['μήθ’'] ="μή θ’"
    
    equivs2['ἤτοι'] ="ἦ τοι"
    equivs2['οἵδε'] ="οἵ δε"
    equivs2['οἵδ’'] ="οἵ δ’"
    equivs2['σύντρεις'] ="σὺν τρεῖς"
    
    equivs2['τάδε'] ="τά δε"
    equivs2['τάδ’'] ="τά δ’"
    equivs2['γρηῢς'] ="γρῆυς"
    equivs2['γρηὺς'] ="γρῆυς"
    
     
    
    
    equivs1['ἀιζηῶν'] ="αἰζηῶν"
    equivs1['ἀίσσονται'] ="ἀΐσσονται"
    equivs1['Ἀτρείδαι'] ="Ἀτρεΐδαι"
    equivs1['Ἀτρείδης'] ="Ἀτρεΐδης"
    equivs1['Ἀτρείδαο'] ="Ἀτρεΐδαο"
    equivs1['ἀύσας'] ="ἀΰσας"

    equivs1['δηιοτῆτα'] ="δηϊοτῆτα"
   
    equivs1['ἄνεω'] ="ἄνεῳ"
    equivs1['Διῒ'] ="Διὶ"
    
     
    equivs1['ἐικυῖα'] ="ἐϊκυῖα"
    
    equivs1['ἠυκόμοιο'] ="ἠϋκόμοιο"
    equivs1['εἴ'] ="ἤ"
    equivs1['ηἴα'] ="ᾖα"
    equivs1['ἤισαν'] ="ἤϊσαν"
    equivs1['Νυσήιον'] ="Νυσήϊον"
 
    equivs1['πάις'] ="πάϊς"
    equivs1['προιάψει'] ="προϊάψει"
    
    equivs1['τῶ'] ="τῷ"
    equivs1['χρυσῇ'] ="χρυσέῃ"
     
 
    

    for l in f:
        if(not re.search('tlg00',l)):
            continue
        l = re.sub('\n$','',l)
        tokencount = tokencount + 1
        args = l.split()
        curform = nodiaeresis(args[1])
        
        workcite = args[0]
        m = re.search('tlg00[12]:([0-9]+)\.([0-9]+)',l)
        if(m):
            curbook = int(m[1])
            curline = int(m[2])
            booklines[curbook] = curline
            
        curkey = workcite + ':' + curform
        if(curkey in metercites):
            m = re.search(':([0-9]+)$',curkey)
            if(m):
                index = int(m[1]) + 1
            else:
                index = 2
            basecit = re.sub(':([0-9]+)$','',curkey)
            curkey = basecit + ':' + str(index)
            #print(curkey)
        metercites[curkey] = 1
        if(not curkey in curcites):
            
            workkey = curkey.lower()
            if( workkey in curcites):
                continue
            workkey = re.sub(':[0-9]+$','',l)
            if( workkey in curcites):
                continue

            curform = re.sub(';','',curform)
            curform = re.sub('·','',curform)
            
            if(curform in equivs1):
                newmatch = equivs1[curform]
                testkey = workcite + ':' + newmatch
                if(testkey in curcites):
                    #print('hit1',curform,testkey,srch,textlines[workcite])
                    continue
            if(curform in equivs2):

                srch = equivs2[curform]
                if(re.search(srch,textlines[workcite])):
                    #print('hit2',srch,curkey,srch,textlines[workcite])
                    continue
            workkey = re.sub('·','',curkey)
            workkey = re.sub(';','',workkey)
            if(not workkey in curcites):
                closest,fuzzn = getbestfit(curkey)
                if(fuzzn == 100):
                    continue
                badcount= badcount + 1
                books = re.sub('tlg00[12]:([0-9]+)\..+','\g<1>',curkey)
                curbook = int(books)
                badbooks[curbook] = badbooks[curbook] + 1
                print(workkey,curkey,closest,fuzzn)
                if(curform in badforms):
                    badforms[curform] = badforms[curform] + 1
                else:
                    badforms[curform] = 1

    print('tokencount',tokencount,'badtokens',badcount,'wordclasses',len(badforms))
    for foo in badbooks:
        print(foo,badbooks[foo],badbooks[foo]/booklines[foo])
    for foo in sorted(badforms,key=badforms.get,reverse=True):
        print(foo,badforms[foo])
    f.close()
checkwords('tlg0012.tlg002.meter.tsv')
#for foo in metercites:
#    if(not foo in curcites):
#        print(foo)

tlg002:1.58:ἀποθρᾐσκοντα tlg002:1.58:ἀποθρᾐσκοντα ἀποθρῴσκοντα 92
tlg002:1.115:πατέρ tlg002:1.115:πατέρ πατέρ’ 91
tlg002:1.143:ἐπᾐχετο tlg002:1.143:ἐπᾐχετο ἐπῴχετο 86
tlg002:1.324:ἐπᾐχετο tlg002:1.324:ἐπᾐχετο ἐπῴχετο 86
tlg002:2.102:κτεατίσσασ’ tlg002:2.102:κτεατίσσασ’ κτεατίσσας 86
tlg002:2.114:ὅτεᾐ tlg002:2.114:ὅτεᾐ ὅτεῴ 75
tlg002:3.166:ὃ12 tlg002:3.166:ὃ12 ὃ 50
tlg002:3.187:κεύσω tlg002:3.187:κεύσω δεύσω 80
tlg002:3.373:ὀφθαλμοῖσιν tlg002:3.373:ὀφθαλμοῖσιν ὀφθαλμοῖς 80
tlg002:3.435:εἰργάζετο tlg002:3.435:εἰργάζετο εἰργάξετο 89
tlg002:4.14:χρυσῆς tlg002:4.14:χρυσῆς χρυσέης 77
tlg002:4.19:ἐξάρχοντες tlg002:4.19:ἐξάρχοντες ἐξάρχοντος 90
tlg002:4.19:μέσσον tlg002:4.19:μέσσον μέσσους 77
tlg002:4.33:φαγόντες tlg002:4.33:φαγόντες φαγόντε 93
tlg002:4.37:ἐκ tlg002:4.37:ἐκ κέκλετο 22
tlg002:4.62:σφῶν tlg002:4.62:σφῶν σφῷν 75
tlg002:4.72:κατὰ tlg002:4.72:κατὰ κὰδ 57
tlg002:4.91:ἔπεφνε tlg002:4.91:ἔπεφνε ἔπεφνεν 92
tlg002:4.95:πόλλ’ tlg002:4.95:πόλλ’ πολλὰ 60
tlg002:4.95:ἔπαθον tlg002:4.95:ἔπαθ

tlg002:8.147:κεν tlg002:8.147:κεν μεῖζον 44
tlg002:8.147:ᾖσιν tlg002:8.147:ᾖσιν ἔῃσιν 67
tlg002:8.155:πόλλ’ tlg002:8.155:πόλλ’ πολλὰ 60
tlg002:8.155:ἔπαθον tlg002:8.155:ἔπαθον πάθον 73
tlg002:8.155:πόλλ’:2 tlg002:8.155:πόλλ’:2 πολλὰ 60
tlg002:8.155:ἐμόγησα tlg002:8.155:ἐμόγησα μόγησα 92
tlg002:8.162:ἔασι tlg002:8.162:ἔασι ἔασιν 89
tlg002:8.174:αὖτ’ tlg002:8.174:αὖτ’ αὖ 67
tlg002:8.175:ἀμφὶ tlg002:8.175:ἀμφὶ ἀμφιπεριστέφεται 30
tlg002:8.175:περιστέφεται tlg002:8.175:περιστέφεται ἀμφιπεριστέφεται 86
tlg002:8.194:ὀνόμαζε tlg002:8.194:ὀνόμαζε ὀνόμαζεν 93
tlg002:8.198:τόν tlg002:8.198:τόν τόδε 57
tlg002:8.201:Φαιήκεσσι tlg002:8.201:Φαιήκεσσι Φαιήκεσσιν 95
tlg002:8.235:προσέειπε tlg002:8.235:προσέειπε προσέειπεν 95
tlg002:8.252:περιγινόμεθ’ tlg002:8.252:περιγινόμεθ’ περιγιγνόμεθ’ 96
tlg002:8.259:ἀγῶνα tlg002:8.259:ἀγῶνα ἀγῶνας 91
tlg002:8.267:ἐυστεφάνου tlg002:8.267:ἐυστεφάνου εὐστεφάνου 80
tlg002:8.268:πρῶτ’ tlg002:8.268:πρῶτ’ πρῶτα 80
tlg002:8.268:ἐμίγησαν tlg002:8.268:ἐμίγησαν μίγησαν 93


tlg002:10.500:φωνήσας tlg002:10.500:φωνήσας δή 22
tlg002:10.500:ἔπεα tlg002:10.500:ἔπεα ἔπεσσιν 55
tlg002:10.500:πτερόεντα tlg002:10.500:πτερόεντα τότε 46
tlg002:10.500:προσηύδων tlg002:10.500:προσηύδων προσέειπον 53
tlg002:10.513:ῥέουσι tlg002:10.513:ῥέουσι ῥέουσιν 92
tlg002:10.518:νεκύεσσι tlg002:10.518:νεκύεσσι νεκύεσσιν 94
tlg002:10.553:ᾖσιν tlg002:10.553:ᾖσιν ᾗσιν 75
tlg002:10.559:κατ’ tlg002:10.559:κατ’ καταντικρὺ 43
tlg002:10.559:ἀντικρὺ tlg002:10.559:ἀντικρὺ καταντικρὺ 71
tlg002:10.568:ἐγίνετο tlg002:10.568:ἐγίνετο ἐγίγνετο 93
tlg002:11.6:κυανοπρώροιο tlg002:11.6:κυανοπρώροιο κυανοπρῴροιο 92
tlg002:11.8:ἐυπλόκαμος tlg002:11.8:ἐυπλόκαμος εὐπλόκαμος 80
tlg002:11.16:ἐπιδέρκεται tlg002:11.16:ἐπιδέρκεται καταδέρκεται 70
tlg002:11.58:ἐὼν tlg002:11.58:ἐὼν ἰὼν 67
tlg002:11.64:κατ’ tlg002:11.64:κατ’ καταντικρὺ 43
tlg002:11.64:ἀντικρὺ tlg002:11.64:ἀντικρὺ καταντικρὺ 71
tlg002:11.70:εὐεργέα tlg002:11.70:εὐεργέα ἐυεργέα 71
tlg002:11.74:ἔστι tlg002:11.74:ἔστι ἔστιν 89
tlg002:11.91:προσέειπε

tlg002:14.351:διήρεσα tlg002:14.351:διήρεσα διήρεσσ’ 80
tlg002:14.366:ὅτ’ tlg002:14.366:ὅτ’ τ’ 80
tlg002:14.374:ἔλθοι tlg002:14.374:ἔλθοι ἔλθῃ 67
tlg002:14.422:ὅ tlg002:14.422:ὅ ὅγ’ 50
tlg002:14.422:γ’ tlg002:14.422:γ’ ὅγ’ 80
tlg002:14.423:θεοῖσι tlg002:14.423:θεοῖσι θεοῖσιν 92
tlg002:14.428:πάντοθεν tlg002:14.428:πάντοθεν πάντων 71
tlg002:14.430:ἄλλα tlg002:14.430:ἄλλα τἆλλα 67
tlg002:14.441:με tlg002:14.441:με τε 50
tlg002:14.471:ἤρχον tlg002:14.471:ἤρχον ἄρχον 80
tlg002:14.481:ἀφραδέως tlg002:14.481:ἀφραδέως ἀφραδίῃς 75
tlg002:14.491:ἐκεῖνος tlg002:14.491:ἐκεῖνος κεῖνος 92
tlg002:15.21:ὀπυίοι tlg002:15.21:ὀπυίοι ὀπυίῃ 73
tlg002:15.23:τεθνηότος tlg002:15.23:τεθνηότος τεθνηκότος 95
tlg002:15.44:ἔγειρε tlg002:15.44:ἔγειρε ἔγειρεν 92
tlg002:15.73:ἐποτρύνῃ tlg002:15.73:ἐποτρύνῃ ἐποτρύνει 82
tlg002:15.73:κατερύκῃ tlg002:15.73:κατερύκῃ κατερύκει 82
tlg002:15.83:ἀμπέμψει tlg002:15.83:ἀμπέμψει ἀππέμψει 88
tlg002:15.89:ἐμοῖσι tlg002:15.89:ἐμοῖσι ἐμοῖσιν 92
tlg002:15.99:κατεβήσατο tlg002:15.99

tlg002:18.264:πτολέμοιο tlg002:18.264:πτολέμοιο πολέμοιο 94
tlg002:18.285:περίφρων tlg002:18.285:περίφρων περίφρον 88
tlg002:18.293:ἄρ’ tlg002:18.293:ἄρ’ ἀρ’ 67
tlg002:18.312:διογενὴς tlg002:18.312:διογενὴς διογενῆς 88
tlg002:18.321:καλλιπάρηος tlg002:18.321:καλλιπάρηος καλλιπάρῃος 91
tlg002:18.326:ἐπέεσσι tlg002:18.326:ἐπέεσσι· ἐπέεσσιν 88
tlg002:18.344:ἑστήκει tlg002:18.344:ἑστήκει ἑστήκειν 93
tlg002:18.350:γέλων tlg002:18.350:γέλων γέλω 89
tlg002:18.375:ὧλκα tlg002:18.375:ὧλκα ὦλκα 75
tlg002:18.381:ὑβρίζεις tlg002:18.381:ὑβρίζεις ὑβρίξεις 88
tlg002:18.387:δὲ tlg002:18.387:δὲ δ’ 50
tlg002:18.387:χολώσατο tlg002:18.387:χολώσατο ἐχολώσατο 94
tlg002:18.402:κ’ tlg002:18.402:κ’ κέλαδον 22
tlg002:18.426:λείψαντες tlg002:18.426:λείψαντες σπείσαντες 74
tlg002:19.10:ἔμβαλε tlg002:19.10:ἔμβαλε ἔβαλε 91
tlg002:19.71:ὦδ’ tlg002:19.71:ὦδ’ ὧδ’ 67
tlg002:19.81:μή tlg002:19.81:μή μήποτε 50
tlg002:19.81:ποτε tlg002:19.81:ποτε μήποτε 80
tlg002:19.91:ἀδδεές tlg002:19.91:ἀδδεές ἀδεές 91
tlg002:19.122:δά

tlg002:22.140:πῃ tlg002:22.140:πῃ πη 50
tlg002:22.143:ἐς tlg002:22.143:ἐς εἰς 40
tlg002:22.146:ὧκα tlg002:22.146:ὧκα ὦκα 67
tlg002:22.146:ἔδωκε tlg002:22.146:ἔδωκε ἔδωκεν 91
tlg002:22.174:ὄπισθεν tlg002:22.174:ὄπισθεν ὄπισθε 92
tlg002:22.186:δ’ tlg002:22.186:δ’ δὴ 50
tlg002:22.186:ἐλέλυντο tlg002:22.186:ἐλέλυντο λέλυντο 93
tlg002:22.209:ὁμηλικίη tlg002:22.209:ὁμηλικίη ὁμηλικίην 94
tlg002:22.215:ὀίω tlg002:22.215:ὀίω· ὀΐω 86
tlg002:22.231:γε tlg002:22.231:γε τε 50
tlg002:22.234:δυσμενέεσσι tlg002:22.234:δυσμενέεσσι δυσμενέεσσιν 96
tlg002:22.243:δαίφρων tlg002:22.243:δαίφρων· δαΐφρων 93
tlg002:22.275:βεβλήκει tlg002:22.275:βεβλήκει βεβλήκειν 94
tlg002:22.280:ὧμον tlg002:22.280:ὧμον ὦμον 75
tlg002:22.288:θεοῖσι tlg002:22.288:θεοῖσι θεοῖσιν 92
tlg002:22.306:γίνεται tlg002:22.306:γίνεται γίγνεται 93
tlg002:22.335:ἕζοιτο tlg002:22.335:ἕζοιτο ἵζοιτο 83
tlg002:22.349:μή tlg002:22.349:μή με 50
tlg002:22.364:ὑπὸ tlg002:22.364:ὑπὸ ἀπὸ 67
tlg002:22.364:ὧρτο tlg002:22.364:ὧρτο ὦρτο 75
tlg002:22.364

ἀντικρὺ 2
ἴδῃς 2
φοινικοπαρήους 2
γήρᾳ 2
κεῖνος 2
ἀνερρύβδησε 2
γίνετ’ 2
ἄλλα 2
ἐκαθῖζον 2
ὄπισθεν 2
πώ 2
οὔ 2
εἰδυίῃ 2
οἱ 2
σφιν 2
τοι 2
ἐπεὶ 2
ἐπακούσῃ 2
καλλιπάρηος 2
τείως 2
ἐζεύγνυντ’ 2
που 2
νηλιτεῖς 2
ὅς 2
πόδας 2
Ὀδυσῆος 2
γίνοντ’ 2
χολώσατο 2
οὐ 2
ὅπῃ 2
πτολέμοιο 2
μή 2
ποτε 2
ἐπὴν 2
μεγάροισι 2
σφ’ 2
εἷλε 2
ὧπα 2
ἀποσταίη 2
εἰδυῖαν 2
σε 2
ἀποθρᾐσκοντα 1
πατέρ 1
κτεατίσσασ’ 1
ὅτεᾐ 1
ὃ12 1
κεύσω 1
ὀφθαλμοῖσιν 1
εἰργάζετο 1
χρυσῆς 1
ἐξάρχοντες 1
μέσσον 1
φαγόντες 1
σφῶν 1
ἔπεφνε 1
χρυσῆν 1
πυκνὸν 1
ἔξοχα 1
πάντων 1
ἀγάσσασθαι 1
ἀνειρώτευν 1
περίστιξας 1
ὁρμηθέντες 1
πίεζε 1
τό 1
ἐστόρεσαν 1
προσέφη 1
ἐλέησε 1
μεθιεῖς 1
ταῦτα 1
ἀμφιβεβήκει 1
γινόμενος 1
ἐπεί 1
θανον 1
ἐπέλασσε 1
κατέπεφνε 1
ἐπὶ 1
εὐήνορα 1
συβώτῃ 1
ἡμῖν 1
πῆμα 1
γενέσθαι 1
δερματίνοισι 1
αὐδήν 1
ἠὼς 1
νηὶ 1
τιμήσουσι 1
ἠνώγει 1
ἔγωγ’ 1
ἐπέθηκε 1
ἐσχατιήν 1
ἤνυτο 1
ἀλλήλοισι 1
ἔπιπτε 1
ἐπιπλείων 1
τάδὲ 1
ἔαξε 1
δυνάσθη 1
ἐκαθῖζε 1
ἀνεδύσατο 1
ηἴων 1
Ὀδυσῆ’ 1
ἐτέλεσσα 1
φέρεν 1
σῴζων 1
ἀννέφελος 1
τοκεῦσι 1
ὅπλεον 

In [9]:
textlines['tlg001:1.27']

'ἢ νῦν δηθύνοντ’ ἢ ὕστερον αὖτις ἰόντα ,'

In [16]:
import re

full_lines = {}
meter_lines = {}
caesuras = {}
caesmasc3 = {}
caesmasc4 = {}
caesfem3 = {}
caesfem4 = {}
bucdiaer = {}
hermannbroken = {}
is_enclitic = {}

is_enclitic['γε'] = 1
is_enclitic['γέ'] = 1
is_enclitic["γ’"] = 1
is_enclitic['κε'] = 1
is_enclitic['κεν'] = 1
is_enclitic['ἑ'] = 1
is_enclitic['κέ'] = 1
is_enclitic["κ’"] = 1
is_enclitic['με'] = 1
is_enclitic['μέ'] = 1
is_enclitic["μ’"] = 1
is_enclitic['μευ'] = 1
is_enclitic['μεύ'] = 1
is_enclitic['μιν'] = 1
is_enclitic['μοι'] = 1
is_enclitic['οἱ'] = 1
is_enclitic["περ"] = 1
is_enclitic["πέρ"] = 1
is_enclitic["ποθ’"] = 1
is_enclitic["ποτ’"] = 1
is_enclitic["ποτε"] = 1
is_enclitic["που"] = 1
is_enclitic["πού"] = 1

is_enclitic["πω"] = 1
is_enclitic["πώ"] = 1
is_enclitic['ῥα'] = 1
is_enclitic['σε'] = 1
is_enclitic['σέ'] = 1
is_enclitic['σευ'] = 1
is_enclitic['σεύ'] = 1
is_enclitic["σ’"] = 1
is_enclitic["σφιν"] = 1
is_enclitic["σφι"] = 1
is_enclitic['τε'] = 1
is_enclitic['τέ'] = 1
is_enclitic["τ’"] = 1
is_enclitic["τι"] = 1
is_enclitic["τινας"] = 1
is_enclitic["τινες"] = 1
is_enclitic["τις"] = 1
is_enclitic["τοι"] = 1
is_enclitic["τοί"] = 1

nocaesurabefore = {}
nocaesurabefore['ἄν'] = 1
nocaesurabefore['ἂν'] = 1
nocaesurabefore['ἄρ’'] = 1

nocaesurabefore['δέ'] = 1
nocaesurabefore['δὲ'] = 1
nocaesurabefore['δ’'] = 1
nocaesurabefore['μέν'] = 1
nocaesurabefore['μὲν'] = 1
nocaesurabefore['γάρ'] = 1
nocaesurabefore['γὰρ'] = 1


nocaesuraafter = {}

nocaesuraafter['ἀλλ’'] = 1
nocaesuraafter['ἀλλὰ'] = 1
nocaesuraafter['ἀλλά'] = 1
nocaesuraafter['καί'] = 1
nocaesuraafter['καὶ'] = 1
wordstarts = {}
for i in range(1,25):
    wordstarts[i] = 0
    
def getlines(fname):
    f = open(fname)
    nofollowingcaes = 0
    outf = open('caesura.'+fname,'w')
    for l in f:

        l = re.sub('\n$','',l)
        if(not l):
            print(l,file=outf)
            continue
        if(re.search('^#',l)):
            print(l,file=outf)
            continue
        args = l.split()
        if(not len(args)==5):
            print(len(args),l)
        curcit = args[0]
        curform = args[1]
        if(curcit in full_lines):
            full_lines[curcit] = full_lines[curcit] + ' ' + curform
        else:
            full_lines[curcit] = curform
        if(curcit in meter_lines):
            meter_lines[curcit] = meter_lines[curcit] + ' ' + args[4]
        else:
            meter_lines[curcit] = args[4]
        #print(args[3])
        startpart = int(args[2])
        if(startpart == 1):
            curhemi = 'hemi1'

        wordstarts[startpart] = wordstarts[startpart] + 1
        endpart = int(args[3])
        wordlen = 0
        workw,wordlen = re.subn('short','',l)
        workw,n = re.subn('long','',l)
        wordlen = wordlen + n * 2
        rightlen = endpart - startpart + 1
        if(wordlen and not rightlen == wordlen):
            
            print(wordlen,l)
        
        if(endpart>24):
            print('start',l)
        if(nofollowingcaes):
            nofollowingcaes = 0
            print(l,curhemi,sep='\t',file=outf)
            continue
        if(curform in nocaesuraafter):
            nofollowingcaes = 1
        if(curform in is_enclitic or curform in nocaesurabefore):
            #print('skipping',curcit,curform)
            print(l,curhemi,sep='\t',file=outf)
            continue
        if(startpart == 11):
            # no caesura after a monosyllabic preposition
            if(curform in prepositions and endpart == 11):
                print(l,curhemi,file=outf)
                continue
            caesmasc3[curcit]  = l
            curhemi = 'hemi2'
            print(l,curhemi,'caesmasc3',sep='\t',file=outf)
            continue
        if(startpart == 12):
            caesfem3[curcit] = l
            curhemi = 'hemi2'
            print(l,curhemi,'caesfem3',sep='\t',file=outf)
            
            continue
        if(startpart == 15):
            caesmasc4[curcit]  = l
            curhemi = 'hemi2'
            print(l,curhemi,'caesmasc4',sep='\t',file=outf)
            continue
        if(startpart == 16):
            curhemi = 'hemi2'
            print(l,curhemi,'caesfem4',sep='\t',file=outf)
            caesfem4[curcit]  = l
            continue
        if(startpart == 17):
            bucdiaer[curcit] = l
            print(l,curhemi,'bucdiaer',sep='\t',file=outf)
            continue
        print(l,curhemi,sep='\t',file=outf)

    f.close()
    outf.close()
    
getlines('tlg0012.tlg001.meter.tsv')
getlines('tlg0012.tlg002.meter.tsv')

def get_meter_stats(s):
    l = meter_lines[s]
    consonants = '[βγδζθκλμνπξρστςφψχ]'
    doublecons = 'short-(.[βγδζθκλμνπξρστςφψχ]\\s*[βγδζθκλμνπξρστςφψχ]+)'

    noelision = '([εαο]\\s(long|short)-[ἐἀὀἔἄὄἠὠἤὤ])'
    #print(l)
    while(re.search(doublecons,l)):
        m = re.search(doublecons,l)
        print('short',s,m[1])
        l = re.sub(doublecons,'',l,1)
        
    while(re.search(noelision,l)):
        m = re.search(noelision,l)
        print(s,'noelis',m[1],full_lines[s])
        l = re.sub(noelision,'',l,1)
        
    synezisis = '(long-ε[ωῶῷῳ])'
    while(re.search(synezisis,l)):
        m = re.search(synezisis,l)
        print(s,'synezisis',m[1],full_lines[s])
        l = re.sub(synezisis,'',l,1)
        
    
    
for foo in sorted(wordstarts,key=wordstarts.get,reverse=True):
    footnum = 1 + (foo-1)*.25
    print(foo,footnum,wordstarts[foo])
caes3 = 0
caes4 = 0

print('caesmasc3',len(caesmasc3))
print('caesfem3',len(caesfem3))
print('caesmasc4',len(caesmasc4))
print('caesfem4',len(caesfem4))

for foo in full_lines:
    get_meter_stats(foo)
    if(foo in caesmasc3 and foo in caesfem3):
        print('m+f',foo,full_lines[foo])
    if(foo in caesmasc3 or foo in caesfem3):
        caes3 = caes3 + 1
        continue
    if(foo in caesmasc4):
        caes4 = caes4 + 1
        continue

    print()
    print(foo)
    if(foo in caesmasc4):
        print('masc4',caesmasc4[foo])
    if(foo in caesfem4):
        print('fem4',caesfem4[foo])
    if(foo in bucdiaer):
        print('bucdiaer',bucdiaer[foo])
    print(full_lines[foo])
    print(meter_lines[foo])

print('done',len(full_lines),'caes3',caes3,caes3/len(full_lines),'caes4',caes4,caes4/len(full_lines))

2 tlg002:3.166	ὃ   12	12	δ-long-ὴ
1 1.0 27792
7 2.5 18408
17 5.0 17125
12 3.75 17023
5 2.0 16647
15 4.5 14244
11 3.5 14144
20 5.75 13879
3 1.5 12828
4 1.75 8775
21 6.0 8508
19 5.5 8506
13 4.0 7131
9 3.0 6979
8 2.75 4606
16 4.75 1321
23 6.5 1025
2 1.25 10
6 2.25 5
10 3.25 2
14 4.25 1
18 5.25 1
22 6.25 0
24 6.75 0
caesmasc3 11662
caesfem3 16114
caesmasc4 12864
caesfem4 207
tlg001:1.1 synezisis long-εω μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος
tlg001:1.7 noelis ε short-ἄ Ἀτρεΐδης τε ἄναξ ἀνδρῶν καὶ δῖος Ἀχιλλεύς
m+f tlg001:1.33 ὣς ἔφατ’ ἔδεισεν δ’ ὃ γέρων καὶ ἐπείθετο μύθῳ

tlg001:1.68
bucdiaer tlg001:1.68	τοῖσι	17	19	τ-long-οῖσ-short-ι
ἤτοι ὅ γ’ ὣς εἰπὼν κατ’ ἄρ’ ἕζετο τοῖσι δ’ ἀνέστη
long-ἤτ-short-οι short-ὅ γ’ long-ὣς long-εἰπ-long-ὼν κ-short-ατ’ short-ἄρ’ long-ἕζ-short-ετ-short-ο τ-long-οῖσ-short-ι δ’ short-ἀν-long-έστ-long-η
tlg001:1.75 noelis ο short-ἄ μῆνιν Ἀπόλλωνος ἑκατηβελέταο ἄνακτος

tlg001:1.101
bucdiaer tlg001:1.101	τοῖσι	17	19	τ-long-οῖσ-short-ι
ἤτοι ὅ γ’ ὣς εἰπὼν κατ’ ἄρ’ ἕζετο το

tlg001:17.610 noelis ο short-ὀ αὐτὰρ ὃ Μηριόναο ὀπάονά θ’ ἡνίοχόν τε
m+f tlg001:17.737 ἄγριος ἠΰτε πῦρ τό τ’ ἐπεσσύμενον πόλιν ἀνδρῶν
m+f tlg001:17.739 ἐν σέλαϊ μεγάλῳ τὸ δ’ ἐπιβρέμει ἲς ἀνέμοιο
tlg001:18.48 noelis α short-ἐ Μαῖρα καὶ Ὠρείθυια ἐϋπλόκαμός τ’ Ἀμάθεια
m+f tlg001:18.53 εἴδετ’ ἀκούουσαι ὅσ’ ἐμῷ ἔνι κήδεα θυμῷ
m+f tlg001:18.56 ἔξοχον ἡρώων ὃ δ’ ἀνέδραμεν ἔρνεϊ ἶσος
tlg001:18.77 noelis α long-ἔ σεῦ ἐπιδευομένους παθέειν τ’ ἀεκήλια ἔργα
m+f tlg001:18.82 ἶσον ἐμῇ κεφαλῇ; τὸν ἀπώλεσα τεύχεα δ’ Ἕκτωρ
tlg001:18.93 synezisis long-εω Πατρόκλοιο δ’ ἕλωρα Μενοιτιάδεω ἀποτίσῃ
tlg001:18.137 noelis ο short-ἄ τεύχεα καλὰ φέρουσα παρ’ Ἡφαίστοιο ἄνακτος
tlg001:18.213 synezisis long-εω αἴ κέν πως σὺν νηυσὶν ἄρεω ἀλκτῆρες ἵκωνται
tlg001:18.287 noelis ε short-ἐ ἦ οὔ πω κεκόρησθε ἐελμένοι ἔνδοθι πύργων;
tlg001:18.293 synezisis long-εω νῦν δ’ ὅτε πέρ μοι ἔδωκε Κρόνου πάϊς ἀγκυλομήτεω
m+f tlg001:18.386 αἰδοίη τε φίλη τε; πάρος γε μὲν οὔ τι θαμίζεις
tlg001:18.411 noelis ο short-ἀ χωλεύων ὑπὸ δὲ κν

tlg002:9.99 noelis α short-ἐ νηυσὶ δ’ ἐνὶ γλαφυρῇσιν ὑπὸ ζυγὰ δῆσα ἐρύσσας
tlg002:9.159 noelis α long-ἐ νῆες μέν μοι ἕποντο δυώδεκα ἐς δὲ ἑκάστην
tlg002:9.194 noelis α short-ἔ αὐτοῦ πὰρ νηΐ τε μένειν καὶ νῆα ἔρυσθαι
tlg002:9.210 noelis α short-ἀ χεῦ’ ὀδμὴ δ’ ἡδεῖα ἀπὸ κρητῆρος ὀδώδει
short tlg002:9.242 υκλ
tlg002:9.263 synezisis long-εω λαοὶ δ’ Ἀτρεΐδεω Ἀγαμέμνονος εὐχόμεθ’ εἶναι
tlg002:9.295 noelis α long-ἔ σχέτλια ἔργ’ ὁρόωντες ἀμηχανίη δ’ ἔχε θυμόν
m+f tlg002:9.374 ψωμοί τ’ ἀνδρόμεοι ὁ δ’ ἐρεύγετο οἰνοβαρείων
tlg002:9.376 noelis ο short-ἔ εἵως θερμαίνοιτο ἔπεσσι δὲ πάντας ἑταίρους
tlg002:9.438 noelis ο long-ἄ καὶ τότ’ ἔπειτα νομόνδ’ ἐξέσσυτο ἄρσενα μῆλα
tlg002:9.440 noelis ο short-ἄ οὔθατα γὰρ σφαραγεῦντο ἄναξ δ’ ὀδύνῃσι κακῇσι
m+f tlg002:9.453 ὀφθαλμὸν ποθέεις τὸν ἀνὴρ κακὸς ἐξαλάωσε
m+f tlg002:9.489 ἐμβαλέειν κώπῃς ἵν’ ὑπὲκ κακότητα φύγοιμεν
m+f tlg002:9.497 εἰ δὲ φθεγξαμένου τευ ἢ αὐδήσαντος ἄκουσε
tlg002:9.505 synezisis long-εω υἱὸν Λαέρτεω Ἰθάκῃ ἔνι οἰκί’ ἔχοντα
tlg002:9.531 sy

tlg002:24.466 noelis α long-ἐ πείθοντ’· αἶψα δ’ ἔπειτ’ ἐπὶ τεύχεα ἐσσεύοντο
m+f tlg002:24.474 εἰπέ μοι εἰρομένῃ τί νύ τοι νόος ἔνδοθι κεύθει;
tlg002:24.494 noelis α short-ἔ αἶψα δ’ Ὀδυσσῆα ἔπεα πτερόεντα προσηύδα·
m+f tlg002:24.545 ὣς φάτ’ Ἀθηναίη ὁ δ’ ἐπείθετο χαῖρε δὲ θυμῷ
done 27793 caes3 27419 0.9865433742309214 caes4 343 0.012341237002122836


In [6]:
import re

f = open('tlg0012.tlg001.meter.tsv')
outf = open('meter.tmp','w')
prevcit = ''
for l in f:
    if(re.search('^#',l)):
        print(l,end='',file=outf)
        continue
    args = l.split()
    curform = args[1]
    startplace = int(args[2])
    endplace = int(args[3])
    meterform = args[4]
    
    curcit = args[0]
    if(not curcit == prevcit):
        diffval = startplace - 1
    startplace = startplace - diffval
    endplace = endplace - diffval
    #if(diffval):
        
    #    print('a',l,end='')
    #    print('b',curcit,curform,startplace,endplace,meterform)
    prevcit = curcit
    print(curcit,curform,startplace,endplace,meterform,sep='\t',file=outf) 

outf.close()
f.close()

In [15]:
len(caesfem4)
caesfem4

{'tlg001:1.20': 'tlg001:1.20\tἄποινα\t16\t19\tδ’-short-ἄπ-long-οιν-short-α',
 'tlg001:1.137': 'tlg001:1.137\tκεν\t16\t16\tκ-short-εν',
 'tlg001:1.168': 'tlg001:1.168\tκάμω\t16\t18\tκ-short-άμ-long-ω',
 'tlg001:1.174': 'tlg001:1.174\tἔμοιγε\t16\t19\tshort-ἔμ-long-οιγ-short-ε',
 'tlg001:1.235': 'tlg001:1.235\tὄρεσσι\t16\t19\tshort-ὄρ-long-εσσ-short-ι',
 'tlg001:1.324': 'tlg001:1.324\tκεν\t16\t16\tκ-short-εν',
 'tlg001:1.350': 'tlg001:1.350\tἀπείρονα\t16\t20\tshort-ἀπ-long-είρ-short-ον-short-α',
 'tlg001:1.576': 'tlg001:1.576\tχερείονα\t16\t20\tχ-short-ερ-long-εί-short-ον-short-α',
 'tlg001:2.239': 'tlg001:2.239\tἀμείνονα\t16\t20\tshort-ἀμ-long-είν-short-ον-short-α',
 'tlg001:2.246': 'tlg001:2.246\tἐὼν\t16\t18\tshort-ἐ-long-ὼν',
 'tlg001:2.250': 'tlg001:2.250\tἔχων\t16\t18\tshort-ἔχ-long-ων',
 'tlg001:2.475': 'tlg001:2.475\tνομῷ\t16\t18\tν-short-ομ-long-ῷ',
 'tlg001:2.479': 'tlg001:2.479\tΠοσειδάωνι\t16\t24\tΠ-short-οσ-long-ειδ-long-ά-long-ων-long-ι',
 'tlg001:2.529': 'tlg001:2.529\tἔην\t

In [5]:
import re

legalmora = []
for i in range(1,7):
    print((i-1)*4  + 1)
    curmora = (i-1)*4  + 1
    legalmora.append(curmora)
    curmora = curmora + 2
    legalmora.append(curmora)
    if(i<6):
        curmora = curmora + 1
        legalmora.append(curmora)

print(legalmora)

1
5
9
13
17
21
[1, 3, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17, 19, 20, 21, 23]


In [21]:
len(full_lines)

27791

In [6]:
def fixmeter(fname):
    f = open(fname)
    fout = open('out.'+fname,'w')
    for l in f:
        l = re.sub('\n','',l)
        if(not re.search('tlg00',l)):
            print(l,file=fout)
            continue
        args = l.split()
        m = re.search('^(σφ’)(.+)$',args[1])
        if(m):
            shortw=m[1]
            l1 = args[0] +'\t' + shortw + '\t' + args[2] + '\t' + args[2] + '\t' + shortw
            longw1 = re.sub(shortw,'',args[1])
            longw2 = re.sub(shortw + '-','',args[4])
            longw2 = re.sub('(long|short)-'+ shortw,'\g<1>-',longw2)
            l2 = args[0] +'\t' + longw1 + '\t' + args[2] + '\t' + args[3] + '\t' + longw2

            print(l1,file=fout)
            print(l2,file=fout)
            print()
            print(l)
            print('1',l1)
            print('2',l2)
        else:
            if(re.search('\t(σφ’)-([^\t]+)$',l)):
                l = re.sub('\t(σφ’)-([^\t]+)$','\t\g<2>',l)
                print('3',l)
            print(l,file=fout)
    
    f.close()
    fout.close()
    
    
fixmeter('tlg0012.tlg001.meter.tsv')
fixmeter('tlg0012.tlg002.meter.tsv')


tlg001:3.300	σφ’ἐγκέφαλος	5	10	σ-long-φ’ἐγκ-short-έ-short-φαλ-long-ος
1 tlg001:3.300	σφ’	5	5	σφ’
2 tlg001:3.300	ἐγκέφαλος	5	10	σ-long-φ’ἐγκ-short-έ-short-φαλ-long-ος

tlg001:8.4	σφ’ἀγόρευε	7	11	σφ’-short-ἀγ-short-όρ-long-ευ-short-ε
1 tlg001:8.4	σφ’	7	7	σφ’
2 tlg001:8.4	ἀγόρευε	7	11	short-ἀγ-short-όρ-long-ευ-short-ε

tlg001:8.412	σφ’ἔννεπε	17	20	σ-long-φ’ἔνν-short-επ-short-ε
1 tlg001:8.412	σφ’	17	17	σφ’
2 tlg001:8.412	ἔννεπε	17	20	σ-long-φ’ἔνν-short-επ-short-ε

tlg001:11.115	σφ’ἦτορ	17	19	σ-long-φ’ἦτ-short-ορ
1 tlg001:11.115	σφ’	17	17	σφ’
2 tlg001:11.115	ἦτορ	17	19	σ-long-φ’ἦτ-short-ορ

tlg001:11.807	σφ’ἀγορή	15	18	σφ’-short-ἀγ-short-ορ-long-ή
1 tlg001:11.807	σφ’	15	15	σφ’
2 tlg001:11.807	ἀγορή	15	18	short-ἀγ-short-ορ-long-ή

tlg001:14.205	σφ’ἄκριτα	13	16	σ-long-φ’ἄκρ-short-ιτ-short-α
1 tlg001:14.205	σφ’	13	13	σφ’
2 tlg001:14.205	ἄκριτα	13	16	σ-long-φ’ἄκρ-short-ιτ-short-α

tlg001:14.304	σφ’ἄκριτα	13	16	σ-long-φ’ἄκρ-short-ιτ-short-α
1 tlg001:14.304	σφ’	13	13	σφ’
2 tlg001:14.304	ἄκριτα	1

In [7]:
def fixmeterlnums(fname):
    f = open(fname)
    fout = open('out.'+fname,'w')
    for l in f:
        l = re.sub('\n','',l)
        if(not re.search('tlg00',l)):
            print(l,file=fout)
            continue
        if(re.search('tlg002:2.388a',l)):
            l = re.sub('tlg002:2.388a','tlg002:2.388',l)
            print(l,file=fout)
            continue
        args = l.split()
        workcit = args[0].split(':')
        #print(l,workcit)
        bookline = workcit[1]
        book,line = bookline.split('.')
        if(workcit[0] == 'tlg001' and book == '8' and int(line) > 548):
            newlnum = int(line) + 4
            oldstart = args[0]
            newstart = workcit[0] + ':' + book + '.' + str(newlnum)
            l = re.sub(oldstart,newstart,l)
            print('fixed',l)
        if(0 and workcit[0] == 'tlg001' and book == '9' and int(line) > 457):
            newlnum = int(line) + 4
            oldstart = args[0]
            newstart = workcit[0] + ':' + book + '.' + str(newlnum)
            l = re.sub(oldstart,newstart,l)
        if(0 and workcit[0] == 'tlg001' and book == '11' and int(line) > 542):
            newlnum = int(line) + 1
            oldstart = args[0]
            newstart = workcit[0] + ':' + book + '.' + str(newlnum)
            l = re.sub(oldstart,newstart,l)
        if(0 and workcit[0] == 'tlg001' and book == '14' and int(line) > 268):
            newlnum = int(line) + 1
            oldstart = args[0]
            newstart = workcit[0] + ':' + book + '.' + str(newlnum)
            l = re.sub(oldstart,newstart,l)
        if(0 and workcit[0] == 'tlg001' and book == '18' and int(line) > 604):
            newlnum = int(line) + 1
            oldstart = args[0]
            newstart = workcit[0] + ':' + book + '.' + str(newlnum)
            l = re.sub(oldstart,newstart,l)
        if(0 and workcit[0] == 'tlg002' and book == '2' and int(line) > 389):
            newlnum = int(line) - 4
            oldstart = args[0]
            newstart = workcit[0] + ':' + book + '.' + str(newlnum)
            l = re.sub(oldstart,newstart,l)
            #print(newl)
            #print(l)

        print(l,file=fout)
    f.close()
    fout.close()
    
    
fixmeterlnums('tlg0012.tlg001.meter.tsv')
#fixmeterlnums('tlg0012.tlg002.meter.tsv')
print('done')

fixed tlg001:8.553	οἳ	1	2	long-οἳ
fixed tlg001:8.553	δὲ	3	3	δ-short-ὲ
fixed tlg001:8.553	μέγα	4	6	μ-short-έγ-long-α
fixed tlg001:8.553	φρονέοντες	7	11	short-φρον-short-έ-long-οντ-short-ες
fixed tlg001:8.553	ἐπὶ	12	14	short-ἐπ-long-ὶ
fixed tlg001:8.553	πτολέμοιο	15	19	πτ-short-ολ-short-έμ-long-οι-short-ο
fixed tlg001:8.553	γεφύρας	20	24	γ-short-ε-long-φύρ-long-ας
fixed tlg001:8.554	εἴατο	1	4	long-εἴ-short-ατ-short-ο
fixed tlg001:8.554	παννύχιοι	5	10	π-long-ανν-short-ύχ-short-ι-long-οι
fixed tlg001:8.554	πυρὰ	11	12	π-short-υρ-short-ὰ
fixed tlg001:8.554	δέ	13	14	δ-long-έ
fixed tlg001:8.554	σφισι	15	16	σ-short-φισ-short-ι
fixed tlg001:8.554	καίετο	17	20	κ-long-αί-short-ετ-short-ο
fixed tlg001:8.554	πολλά	21	24	π-long-ολλ-long-ά
fixed tlg001:8.555	ὡς	1	2	long-ὡς
fixed tlg001:8.555	δ’	3	3	δ’
fixed tlg001:8.555	ὅτ’	3	3	short-ὅτ’
fixed tlg001:8.555	ἐν	4	4	short-ἐν
fixed tlg001:8.555	οὐρανῷ	5	8	long-οὐρ-short-αν-short-ῷ
fixed tlg001:8.555	ἄστρα	9	11	long-ἄστρ-short-α
fixed tlg001:8.555	φαεινὴν	